In [ ]:
# Install shap if not available
import sys
!{sys.executable} -m pip install shap

# What does this model do?

1. Predicts utilization counts for each category (e.g., PCP visits, Rx fills, inpatient stays).

2. Multiplies predicted counts by unit costs from insurance data to estimate total allowed cost.
This is then run through a benefit calculator to estimate patient out-of-pocket (OOP) costs.

Model is trained on MEPS data: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files.jsp

# Imports

In [1]:
import pandas as pd
import polars as pl # using polars library for large .xlsx files
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.preprocessing import FunctionTransformer

import shap
import joblib

# Load MEPS Data

ChatGPT: Use consolidated from year t and counts from year t+1 if you're predicting future utilization (prospective), which is better for real-world use.

We are using consolidated from 2022 and counts from 2023.

## Consolidated Data

**Skip if h243.xlsx has already been converted to .csv and .parquet.**

In [ ]:
# Define schema for problematic columns
schema_overrides = {
    'HRWG53X': pl.Float64,  # Use float instead of int for decimal values
    # Add other problematic columns as needed
}

df = pl.read_excel(
    "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h243.xlsx",
    schema_overrides=schema_overrides
)

df.head()

Convert to faster data formats for later use

In [ ]:
df.write_csv("/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h243.csv")  # Much faster for future loads
df.write_parquet("/Users/orenj/Desktop/Projects/open-coverage/python-models/data/h243.parquet")  # Even faster, compressed

Load MEPS Consolidated File (person-level features + target)

In [4]:
df_cons = pd.read_parquet("/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2022-consolidated/h243.parquet")

df_cons.head()

,DUID,PID,DUPERSID,PANEL,DATAYEAR,FAMID31,FAMID42,FAMID53,FAMID22,FAMIDYR,...,RXOSR22,RXPTR22,RXOTH22,PERWT22F,FAMWT22F,FAMWT22C,SAQWT22F,DIABW22F,VARSTR,VARPSU
0,2460002,101,2460002101,24,2022,A,A,A,A,A,...,0,0,0,5728.309495,5232.211986,5232.211986,3994.687140,6034.636755,2082,1
1,2460006,101,2460006101,24,2022,A,A,A,A,A,...,0,0,0,15648.881461,16017.881691,16017.881691,0.000000,0.000000,2001,4
2,2460006,102,2460006102,24,2022,A,A,A,A,A,...,0,0,0,14123.720178,16017.881691,12580.731740,0.000000,0.000000,2001,4
3,2460010,101,2460010101,24,2022,A,A,A,A,A,...,0,5288,299,16982.054917,21905.758877,21905.758877,0.000000,0.000000,2038,3
4,2460018,101,2460018101,24,2022,A,A,A,A,A,...,0,10,0,10682.619947,11344.291012,11344.291012,17152.439412,0.000000,2041,1


## Aggregate Event Files

Load data files for **Counts per Person**

In [5]:
event_files = {
    "pcp_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248g.xlsx",            # Office-based visits
    "outpatient_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248f.xlsx",     # Outpatient visits
    "er_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248e.xlsx",             # Emergency Room visits
    "inpatient_admits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248d.xlsx",      # Inpatient stays
    "home_health_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248h.xlsx",    # Home health visits
    "rx_fills": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248a.xlsx",              # Prescription fills
    "dental_visits": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248b.xlsx",         # Dental visits
    "equipment_purchases": "/Users/orenj/Desktop/Projects/open-coverage/python-models/data/MEPS/2023-counts/h248c.xlsx"    # Medical equipment/supplies
}

# Count records per DUPERSID for each event type
counts_dict = {}
for feat_name, filepath in event_files.items():
    # Use pandas with usecols parameter (which works correctly)
    df_evt = pd.read_excel(filepath, usecols=["DUPERSID"])
    
    # Count occurrences per DUPERSID
    counts = df_evt.groupby("DUPERSID").size().rename(feat_name)
    counts_dict[feat_name] = counts

# Merge all counts into a single DataFrame
counts_df = (
    pd.concat(counts_dict.values(), axis=1)
      .fillna(0)
      .reset_index()
)

## Merge consolidated with counts

Doing a left join on DUPERSID so everyone is included in the dataset. Also need to predict those with zero events.

DUPERSID: AHRQ‑generated person identifier (derived from dwelling/unit and person IDs). It uniquely identifies a respondent within a year, and from 2018 onward it’s constructed to be unique across panels/year.

In [6]:
df = (
    df_cons
      .merge(counts_df, on="DUPERSID", how="left")
      .fillna(0)
)
df.head()

,DUID,PID,DUPERSID,PANEL,DATAYEAR,FAMID31,FAMID42,FAMID53,FAMID22,FAMIDYR,...,VARSTR,VARPSU,pcp_visits,outpatient_visits,er_visits,inpatient_admits,home_health_visits,rx_fills,dental_visits,equipment_purchases
0,2460002,101,2460002101,24,2022,A,A,A,A,A,...,2082,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2460006,101,2460006101,24,2022,A,A,A,A,A,...,2001,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2460006,102,2460006102,24,2022,A,A,A,A,A,...,2001,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2460010,101,2460010101,24,2022,A,A,A,A,A,...,2038,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2460018,101,2460018101,24,2022,A,A,A,A,A,...,2041,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.shape

(22431, 1428)

# Select Features and Target Variables

Introduced 30 more features in v2. Feature descriptions are available in table below.

| **Category**               | **Feature**                 | **Variable Name**                  | **Description**                                                    |
| -------------------------- | --------------------------- | ---------------------------------- | ------------------------------------------------------------------ |
| **Identifiers**            | Person ID                   | `person_id`                        | Unique person identifier (for merges)                              |
| **Demographics**           | Age                         | `age_years_2022`                   | Age as of 12/31/2022 (imputed)                                     |
|                            | Sex                         | `gender`                           | Survey-reported gender (1 = Male, 2 = Female)                      |
|                            | Race/Ethnicity              | `race_ethnicity`                   | Detailed race and ethnicity categories                             |
|                            | Hispanic Category           | `hispanic_origin_category`         | Categorical breakdown of Hispanic origin                           |
|                            | Region                      | `census_region`                    | Census region (Northeast, Midwest, South, West)                    |
|                            | Metro Status                | `metro_status`                     | 1 = MSA • 2 = non-MSA • 3 = unknown                                |
| **Socioeconomic**          | Education (years)           | `education_years`                  | Years of schooling when first entered MEPS                         |
|                            | Highest Degree              | `highest_degree_achieved`          | Highest educational degree achieved                                |
|                            | Family Income               | `family_income_2022`               | Total family income in 2022                                        |
|                            | Poverty Level (%)           | `poverty_level_pct`                | Family income as % of federal poverty line                         |
|                            | Poverty Category            | `poverty_category`                 | Poverty status category (<100%, 100–199%, ≥200%)                   |
|                            | Employment Status           | `employment_status`                | Employed full-/part-time vs. unemployed/inactive                   |
|                            | Hours Worked/Week           | `hours_worked_per_week`            | Usual hours worked/week                                            |
|                            | Industry Category           | `occupation_industry_category`     | Broad industry category of current/most recent job                 |
|                            | Family Size                 | `family_size`                      | Number of people in the family unit                                |
|                            | Marital Status              | `marital_status`                   | Married vs. single/divorced/widowed                                |
|                            | Spouse Present              | `spouse_in_household`              | Indicates if spouse resides in same household                      |
| **Access & Utilization**   | Insurance Status            | `insurance_coverage_type`          | 1 = Any private • 2 = Public only • 3 = Uninsured                  |
|                            | Usual Source of Care        | `has_usual_source_of_care`         | Has a usual source of medical care                                 |
|                            | Location of Usual Care      | `usual_care_location_type`         | Type/location of usual source (clinic, doctor’s office, etc.)      |
|                            | Could Not Afford Care       | `delayed_care_due_to_cost`         | Delayed/forgone medical care due to cost                           |
|                            | Delayed Medicine (Cost)     | `delayed_prescription_due_to_cost` | Delayed getting prescription medicine because of cost              |
|                            | Food Stamp Receipt          | `received_snap`                    | Received SNAP benefits                                             |
|                            | Food Stamp Value            | `snap_benefit_value_2022`          | Total value of SNAP benefits received in 2022                      |
| **Behaviors & Lifestyle**  | BMI                         | `bmi`                              | Body Mass Index (>17) from Round 4/5                               |
|                            | Smoking Frequency           | `smoking_frequency`                | “How often use tobacco” in past 12 months                          |
|                            | Alcohol Consumption         | `alcohol_consumption_frequency`    | Alcohol drinking in past 12 months                                 |
|                            | Exercise                    | `exercise_days_per_week`           | Average days/week of moderate/vigorous activity                    |
|                            | English Proficiency         | `english_proficiency`              | Self-rated English speaking ability                                |
|                            | Nativity                    | `us_born_flag`                     | Born in the United States?                                         |
|                            | Years in U.S.               | `years_in_us`                      | Years lived in the United States                                   |
| **Functional & Cognitive** | Difficulty Lifting          | `difficulty_lifting_carrying`      | Serious difficulty lifting or carrying                             |
|                            | Difficulty Walking/Stairs   | `difficulty_walking_stairs`        | Serious difficulty walking or climbing stairs                      |
|                            | Any Activity Limitation     | `any_activity_limitation`          | Any limitation in usual activities due to health                   |
|                            | Cognitive Limitation        | `cognitive_limitation`             | Serious difficulty concentrating, remembering, or making decisions |
|                            | Psychological Distress (K6) | `k6_distress_score`                | Kessler 6-item psychological distress score (0–24)                 |
|                            | Feelings of Hopelessness    | `hopelessness_frequency_30d`       | Frequency of feeling hopeless in past 30 days                      |
|                            | Feelings of Sadness         | `sadness_frequency_30d`            | Frequency of feeling sad or depressed in past 30 days              |
| **Health Status**          | Chronic Condition Count     | `chronic_condition_count`          | Number of chronic conditions                                       |
| **Outcome**                | Total Expenditures (target) | `total_expenditures_2022`          | Sum of all payments in 2022                                        |


### Rename input variables

In [8]:
# Rename MEPS columns to descriptive names
df.rename(columns={
    "DUPERSID": "person_id",
    "AGE22X": "age_years_2022",
    "ADSEX42": "gender",
    "RACETHX": "race_ethnicity",
    "HISPNCAT": "hispanic_origin_category",
    "REGION22": "census_region",
    "MSA22": "metro_status",
    "EDUCYR": "education_years",
    "HIDEG": "highest_degree_achieved",
    "FAMINC22": "family_income_2022",
    "POVLEV22": "poverty_level_pct",
    "POVCAT22": "poverty_category",
    "EMPST42": "employment_status",
    "HOUR42": "hours_worked_per_week",
    "OCCCAT42": "occupation_industry_category",
    "FAMSZE22": "family_size",
    "MARRY22X": "marital_status",
    "SPOUIN22": "spouse_in_household",
    "INSCOV22": "insurance_coverage_type",
    "HAVEUS42": "has_usual_source_of_care",
    "LOCATN42": "usual_care_location_type",
    "AFRDCA42": "delayed_care_due_to_cost",
    "DLAYPM42": "delayed_prescription_due_to_cost",
    "FOODST22": "received_snap",
    "FOODVL22": "snap_benefit_value_2022",
    "ADBMI42": "bmi",
    "ADOSTP42": "smoking_frequency",
    "ADASKALC42": "alcohol_consumption_frequency",
    "ADDAYEXER42": "exercise_days_per_week",
    "HWELLSPK": "english_proficiency",
    "BORNUSA": "us_born_flag",
    "YRSINUS": "years_in_us",
    "LFTDIF31": "difficulty_lifting_carrying",
    "DFWLKC42": "difficulty_walking_stairs",
    "ACTLIM31": "any_activity_limitation",
    "COGLIM31": "cognitive_limitation",
    "K6SUM42": "k6_distress_score",
    "ADHOPE42": "hopelessness_frequency_30d",
    "ADSAD42": "sadness_frequency_30d",
    "TOTCHRON": "chronic_condition_count",
    "TOTEXP22": "total_expenditures_2022"
}, inplace=True)

### Combine datasets

In [9]:
feature_cols = [
    "age_years_2022",
    "gender",
    "race_ethnicity",
    "hispanic_origin_category",
    "census_region",
    "education_years",
    "highest_degree_achieved",
    "family_income_2022",
    "poverty_level_pct",
    "poverty_category",
    "employment_status",
    "hours_worked_per_week",
    "occupation_industry_category",
    "family_size",
    "marital_status",
    "spouse_in_household",
    "insurance_coverage_type",
    "has_usual_source_of_care",
    "usual_care_location_type",
    "delayed_care_due_to_cost",
    "delayed_prescription_due_to_cost",
    "received_snap",
    "snap_benefit_value_2022",
    "bmi",
    "smoking_frequency",
    "alcohol_consumption_frequency",
    "exercise_days_per_week",
    "english_proficiency",
    "us_born_flag",
    "years_in_us",
    "difficulty_lifting_carrying",
    "difficulty_walking_stairs",
    "any_activity_limitation",
    "cognitive_limitation",
    "k6_distress_score",
    "hopelessness_frequency_30d",
    "sadness_frequency_30d",
    # "metro_status",            # couldn't find but doesn't matter for now
    # "chronic_condition_count"  # MAKE SURE TO UNCOMMENT WHEN USING
]

count_targets = [
    "pcp_visits", "outpatient_visits", "er_visits",
    "inpatient_admits", "home_health_visits",
    "rx_fills", "dental_visits", "equipment_purchases"
]

X = df[feature_cols]
y = df[count_targets].clip(lower=0)

## Null Value Analysis

In [ ]:
# df = the raw feature frame BEFORE your existing Pipeline
n_rows = len(df)

missing_summary = (
    df.isna()
      .agg(['sum', 'mean'])            # absolute count + proportion
      .T
      .rename(columns={'sum':'n_null', 'mean':'null_rate'})
      .assign(datatype=df.dtypes)      # keep data type in view
      .sort_values('null_rate', ascending=False)
)

pd.set_option('display.max_rows', None)
display(missing_summary)
# …then later, if you want to go back to default:
pd.reset_option('display.max_rows')

**Conclusion: there are no null values in MEPS data.**

# Preprocess Data

## Train/Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Preprocessor for Categorical Features

The MEPS dataset has reserved negative codes to indicate different reasons for null values instead of just leaving the observation null. For v2 I will just convert these negative codes to null. This can be addressed in a later model version. Ex. -7 = respondant refused to answer the question, -8 = respondant did not know the information

In [11]:
# All negative “not-real” answers in the 2022 Consolidated PUF
MEPS_2022_RESERVED = {
    -1:  "inapplicable",
    -2:  "determined_prev_round",
    -7:  "refused",
    -8:  "dont_know_or_not_ascertained",
    -10: "hourly_wage_topcode",        # wage-only variables
    -13: "initial_wage_imputed",       # wage-only variables
    -15: "cannot_be_computed"
}
NEG_CODES = list(MEPS_2022_RESERVED.keys())

def neg_to_nan(X: pd.DataFrame) -> pd.DataFrame:
    """Replace MEPS reserved negative codes with NaN."""
    return X.replace(NEG_CODES, np.nan)

nanifier = FunctionTransformer(neg_to_nan, feature_names_out="one-to-one") # convert codes to NaN

Process categories using one hot encoding.

In [12]:
# 1. List your categorical columns
categorical_cols = [
    # Demographics
    "gender",
    "race_ethnicity",
    "hispanic_origin_category",
    "census_region",
    #"metro_status", 
    # Socioeconomic (categorical)
    "highest_degree_achieved",
    "poverty_category",
    "employment_status",
    "occupation_industry_category",
    "marital_status",
    "spouse_in_household",
    # Access & Utilization
    "insurance_coverage_type",
    "has_usual_source_of_care",
    "usual_care_location_type",
    "delayed_care_due_to_cost",
    "delayed_prescription_due_to_cost",
    "received_snap",
    # Behaviors & Lifestyle
    "smoking_frequency",
    "alcohol_consumption_frequency",
    "english_proficiency",
    "us_born_flag",
    # Functional & Cognitive
    "difficulty_lifting_carrying",
    "difficulty_walking_stairs",
    "any_activity_limitation",
    "cognitive_limitation",
    # (you can add more here if you treat them as categorical)
]

# 2. Build a little pipeline just for categoricals
categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(      # not imputing NaNs because HistGradientBoostingRegressor natively handles them; change if algorithm changes
        handle_unknown="ignore",   # don't crash if new categories appear in test
        sparse_output=False,             # return a dense array (needed for shap, etc.)
        drop="if_binary"          # drop one level for binary vars to avoid collinearity
    ))
])

# 3. Hook it into a ColumnTransformer (you’d add your numeric step(s) too)
preprocessor = ColumnTransformer([
    ("cat", categorical_transformer, categorical_cols),
    # ("num", numeric_transformer, numeric_cols),  <-- your existing numeric step here
],
    remainder="passthrough"   # if you have any extras you’re just passing through
)

# Train/Fit the Model

## Fit one Poisson GBM per count target

There are **8 count targets** so we are building a model for each.

The HistGradientBoostingRegressor has native support for missing values (NaNs). If you decide to use a different estimator, check to see if you need to imput values.

In [13]:
models = {}
preds = pd.DataFrame(index=X_test.index)

for target in count_targets:
    pipe = Pipeline([
        ("nanifier", nanifier), # convert negative codes to NaN
        ("preproc", preprocessor), # encoding/scaling
        ("model", HistGradientBoostingRegressor(
            loss="poisson",
            max_iter=200,
            learning_rate=0.1,
            random_state=42
        ))
    ])
    pipe.fit(X_train, y_train[target])
    models[target] = pipe
    preds[target] = pipe.predict(X_test).clip(min=0)

# Persist to disk so you can reload later in the same notebook
for target, pipe in models.items():
    joblib.dump(pipe, f"/Users/orenj/Desktop/Projects/open-coverage/python-models/models/v2-pkl/model_{target}.pkl")

Ensure negative MEPS values have been mapped to NaNs. *See Preprocessor for Categorical Features section.*

In [14]:
# Get the fitted OneHotEncoder
ohe = pipe.named_steps['preproc'].named_transformers_['cat'].named_steps['onehot']

# Show the list of categories for each categorical column
for col, cats in zip(categorical_cols, ohe.categories_):
    print(f"{col}: {list(cats)}")

gender: [1.0, 2.0, nan]
race_ethnicity: [1, 2, 3, 4, 5]
hispanic_origin_category: [1, 2, 3, 4, 5, 6, 8, 9]
census_region: [1.0, 2.0, 3.0, 4.0, nan]
highest_degree_achieved: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, nan]
poverty_category: [1, 2, 3, 4, 5]
employment_status: [1.0, 2.0, 3.0, 4.0, nan]
occupation_industry_category: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 11.0, nan]
marital_status: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan]
spouse_in_household: [1.0, 2.0, 3.0, nan]
insurance_coverage_type: [1, 2, 3]
has_usual_source_of_care: [1.0, 2.0, nan]
usual_care_location_type: [1.0, 2.0, 3.0, nan]
delayed_care_due_to_cost: [1.0, 2.0, nan]
delayed_prescription_due_to_cost: [1.0, 2.0, nan]
received_snap: [1.0, 2.0, nan]
smoking_frequency: [1.0, 2.0, nan]
alcohol_consumption_frequency: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, nan]
english_proficiency: [1.0, 2.0, 3.0, 4.0, 5.0, nan]
us_born_flag: [1.0, 2.0, nan]
difficulty_lifting_carrying: [1.0, 2.0, 3.0, 4.0, nan]
difficulty_walking_stairs: [1.

# Model Evaluation

## Evaluate Each Count Model

In [14]:
metrics = []
for target in count_targets:
    mae = mean_absolute_error(y_test[target], preds[target])
    rmse = root_mean_squared_error(y_test[target], preds[target])
    metrics.append({
        "feature": target,
        "MAE": mae,
        "RMSE": rmse
    })

metrics_df = pd.DataFrame(metrics).set_index("feature")
print("Count Models Evaluation (per-category):")
print(metrics_df)

Count Models Evaluation (per-category):
                          MAE       RMSE
feature                                 
pcp_visits           3.853706   8.767582
outpatient_visits    0.675180   3.600926
er_visits            0.145886   0.420662
inpatient_admits     0.070335   0.237845
home_health_visits   0.275654   1.761036
rx_fills             5.763608  12.988660
dental_visits        0.589623   1.105232
equipment_purchases  0.339166   0.607119


## Predict counts for a synthetic user

For testing missing data use 'np.nan' instead of the value.

In [ ]:
# Define realistic synthetic data for one individual
synthetic_user = {
    # Identifiers
    "person_id": 99999999,

    # Demographics
    "age_years_2022": 57,
    "gender": np.nan,                      # Female
    "race_ethnicity": 3,             # Black/African American
    "hispanic_origin_category": -1,  # Not Hispanic
    "census_region": 3,              # South
    # "metro_status": 1,             # In MSA

    # Socioeconomic
    "education_years": 12,
    "highest_degree_achieved": 3,    # High school graduate
    "family_income_2022": 32000,
    "poverty_level_pct": 140,        # 140% of FPL
    "poverty_category": 2,           # 100–199%
    "employment_status": 2,          # Unemployed
    "hours_worked_per_week": 0,
    "occupation_industry_category": -1,  # Not applicable / unemployed
    "family_size": 3,
    "marital_status": 2,             # Widowed
    "spouse_in_household": 2,        # No spouse in household

    # Access & Utilization
    "insurance_coverage_type": 2,          # Public only (e.g., Medicaid)
    "has_usual_source_of_care": 1,         # Has usual source of care
    "usual_care_location_type": 2,         # Clinic/health center
    "delayed_care_due_to_cost": 1,         # Could not afford care
    "delayed_prescription_due_to_cost": 1, # Delayed meds due to cost
    "received_snap": 1,                    # Received food stamps
    "snap_benefit_value_2022": 1500,

    # Behaviors & Lifestyle
    "bmi": 34.2,                         # Obese
    "smoking_frequency": 2,             # Some days
    "alcohol_consumption_frequency": 3, # <4 times/week
    "exercise_days_per_week": 1,        # 1 day/week
    "english_proficiency": 4,           # Very well
    "us_born_flag": 1,                  # Born in US
    "years_in_us": -1,                  # N/A (born in US)

    # Functional & Cognitive
    "difficulty_lifting_carrying": 1,       # Yes, difficulty lifting
    "difficulty_walking_stairs": 2,         # No difficulty walking
    "any_activity_limitation": 1,           # Yes, limited activities
    "cognitive_limitation": 2,              # No cognitive limitations
    "k6_distress_score": 9,                 # Moderate psychological distress
    "hopelessness_frequency_30d": 3,        # Some of the time
    "sadness_frequency_30d": 3,             # Some of the time

    # Health Status
    # "chronic_condition_count": 4,        # Has multiple chronic conditions

    # Outcome (for training)
    "total_expenditures_2022": 18934.65     # Total expenditures in 2022 (USD)
}

# Create DataFrame with the synthetic individual
synthetic_df = pd.DataFrame([synthetic_user])

# Make predictions using the trained models
predictions = {}
for target in count_targets:
    # Use the trained model to predict
    pred_value = models[target].predict(synthetic_df)[0]
    predictions[target] = max(0, round(pred_value))  # Ensure non-negative integers

# Create predictions DataFrame
predictions_df = pd.DataFrame([predictions])

print("Predicted Healthcare Utilization Counts:")
display(predictions_df)

# Feature Analysis

## SHAP Values for Feature Importance

### SHAP v2

Computation in SHAP v1 took an hour to compute - this is uncessary. I am using a technique from this [paper](https://parthenonfrontiers.com/index.php/ejeai/article/view/49) to reduce the computation time. It uses Slovin's sample method to reduce the computational load.

In [112]:
def slovins_sample(df, error=0.02, random_state=42):
    """Return a Slovin-sized subsample of df."""
    N = len(df)
    n = int(N / (1 + N * error**2))
    return df.sample(n=n, random_state=random_state)

Reload a model & build SHAP background

In [ ]:
TARGET = count_targets[7]            # pick a target to explain (0-7)
pipe   = joblib.load(f"/Users/orenj/Desktop/Projects/open-coverage/python-models/models/v2-pkl/model_{TARGET}.pkl")

# Build a reduced background set once
X_background = slovins_sample(X_train, error=0.02)
Xb_trans     = pipe.named_steps["preproc"].transform(X_background)

# TreeExplainer on the fitted tree model
explainer = shap.TreeExplainer(
    pipe.named_steps["model"],       # the trained HGBRegressor
    data=Xb_trans,                    # background in transformed space
    feature_perturbation="interventional" # This forces SHAP to exactly satisfy additivity (at the cost of some speed):
)

# Pre‐transform test set
Xt_trans = pipe.named_steps["preproc"].transform(X_test)

# Compute SHAP‐values on the transformed data, disabling a check
shap_vals = explainer.shap_values(
    Xt_trans,
    check_additivity=False
)

# Plot
feat_names = pipe.named_steps["preproc"].get_feature_names_out()

In [ ]:
shap.summary_plot(
    shap_vals,           # your array of shap values
    Xt_trans,            # the corresponding transformed data
    feature_names=feat_names,
    plot_type="bar",      # <-- this makes it a bar chart
    max_display=5        # show top 5 features
)

Aggregate SHAP models for all 8 models

In [ ]:
# 1) Precompute feature names once
feat_names = models[count_targets[0]].named_steps["preproc"].get_feature_names_out()

# 2) Dict to hold each model's top5 DataFrame
top5_per_model = {}

for tgt in count_targets:
    pipe      = models[tgt]
    # build explainer (interventional) on a common background
    Xb = slovins_sample(X_train, error=0.02)
    Xb_t = pipe.named_steps["preproc"].transform(Xb)
    expl = shap.TreeExplainer(pipe.named_steps["model"], data=Xb_t, feature_perturbation="interventional")
    
    # get shap values for this target
    Xt_t     = pipe.named_steps["preproc"].transform(X_test)
    shap_vals= expl.shap_values(Xt_t, check_additivity=False)
    
    # compute mean absolute shap per feature
    mean_abs = np.abs(shap_vals).mean(axis=0)
    
    # build a sorted DataFrame of top 5
    df = pd.DataFrame({
        "feature":     feat_names,
        "mean_abs_shap": mean_abs
    }).sort_values("mean_abs_shap", ascending=False).head(5).reset_index(drop=True)
    
    df["target"] = tgt
    top5_per_model[tgt] = df

# concatenate into one big table
top5_df = pd.concat(top5_per_model.values(), ignore_index=True)
display(top5_df)

While the one-hot dummy was included in these it is okay to just include the whole variable for now.

18 Most Important features:

| Category                   | Final variable name            | Plain-English feature                |
| -------------------------- | ------------------------------ | ------------------------------------ |
| **Demographics**           | `age_years_2022`               | Age                                  |
|                            | `gender`                       | Sex                                  |
|                            | `race_ethnicity`               | Race / Ethnicity                     |
|                            | `census_region`                | Census Region                        |
| **Socioeconomic**          | `education_years`              | Years of Schooling                   |
|                            | `family_size`                  | Family Size                          |
|                            | `family_income_2022`           | Family Income (2022 \$)              |
|                            | `poverty_level_pct`            | Poverty Level (% FPL)                |
|                            | `occupation_industry_category` | Industry Category                    |
|                            | `employment_status`            | Employment Status                    |
| **Access & Utilization**   | `usual_care_location_type`     | Location of Usual Care               |
|                            | `has_usual_source_of_care`     | Has a Usual Source of Care           |
|                            | `insurance_coverage_type`      | Insurance Status                     |
| **Behaviors & Lifestyle**  | `english_proficiency`          | English-speaking Ability             |
|                            | `bmi`                          | Body-Mass Index                      |
| **Functional & Cognitive** | `difficulty_walking_stairs`    | Difficulty Walking / Climbing Stairs |
|                            | `any_activity_limitation`      | Any Activity Limitation              |
|                            | `k6_distress_score`            | Psychological Distress (K-6)         |


## Baseline Analysis

In [28]:
# Define realistic synthetic data for one individual
baseline_user = {
    # Identifiers
    "person_id": 99999999,

    # Important 
    "age_years_2022": 80,               
    "gender": 1,                            # Male
    "race_ethnicity": 1,                    # Hispanic
    "family_size": 3,
    "family_income_2022": 32000,
    "employment_status": 1,                 # Unemployed
    "bmi": 18.5,                            # Obese
    "difficulty_walking_stairs": 2,         # No difficulty walking
    "any_activity_limitation": 1,           # Yes, limited activities
    "k6_distress_score": 4,                 # Moderate psychological distress

    # Socioeconomic
    "education_years": np.nan,
    "highest_degree_achieved": np.nan,    # High school graduate
    "poverty_level_pct": np.nan,        # 140% of FPL
    "poverty_category": np.nan,           # 100–199%
    "hours_worked_per_week": np.nan,
    "occupation_industry_category": np.nan,  # Not applicable / unemployed
    "marital_status": np.nan,             # Widowed
    "spouse_in_household": np.nan,        # No spouse in household

    # Access & Utilization
    "insurance_coverage_type": np.nan,          # Public only (e.g., Medicaid)
    "has_usual_source_of_care": np.nan,         # Has usual source of care
    "usual_care_location_type": np.nan,         # Clinic/health center
    "delayed_care_due_to_cost": np.nan,         # Could not afford care
    "delayed_prescription_due_to_cost": np.nan, # Delayed meds due to cost
    "received_snap": np.nan,                    # Received food stamps
    "snap_benefit_value_2022": np.nan,

    # Behaviors & Lifestyle
    "smoking_frequency": np.nan,             # Some days
    "alcohol_consumption_frequency": np.nan, # <4 times/week
    "exercise_days_per_week": np.nan,        # 1 day/week
    "english_proficiency": np.nan,           # Very well
    "us_born_flag": np.nan,                  # Born in US
    "years_in_us": np.nan,                  # N/A (born in US)
    #"metro_status": np.nan,                  # N/A (born in US)

    # Functional & Cognitive
    "difficulty_lifting_carrying": np.nan,       # Yes, difficulty lifting
    "cognitive_limitation": np.nan,              # No cognitive limitations
    "hopelessness_frequency_30d": np.nan,        # Some of the time
    "sadness_frequency_30d": np.nan,             # Some of the time

    # Health Status
    #"chronic_condition_count": np.nan,        # Has multiple chronic conditions; CHANGE IN LATER MODEL VERSION

    # Outcome (for training)
    "total_expenditures_2022": 18934.65     # Total expenditures in 2022 (USD)
}

# Create DataFrame with the synthetic individual
baseline_df = pd.DataFrame([baseline_user])

# Make predictions using the trained models
predictions = {}
for target in count_targets:
    # Use the trained model to predict
    pred_value = models[target].predict(baseline_df)[0]
    predictions[target] = max(0, round(pred_value))  # Ensure non-negative integers

# Create predictions DataFrame
predictions_df = pd.DataFrame([predictions])

print("Predicted Healthcare Utilization Counts:")
display(predictions_df)

Predicted Healthcare Utilization Counts:


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:2

,pcp_visits,outpatient_visits,er_visits,inpatient_admits,home_health_visits,rx_fills,dental_visits,equipment_purchases
0,2,0,0,1,0,1,0,0
